# Data Preparation

In this step we load raw CSVs, merge them, clean missing values, and create the target variable `liquidity_ratio = 24h_volume / mkt_cap`.  
We save the cleaned dataset into `data/processed/merged_coin_gecko.csv`.


In [1]:
import pandas as pd
import numpy as np
from pathlib import Path

BASE = Path.cwd().parent if Path.cwd().name=="notebooks" else Path.cwd()
RAW = BASE/"data"/"raw"

# 1) Read all CSVs and merge
files = sorted(RAW.glob("*.csv"))
dfs = []
for p in files:
    d = pd.read_csv(p, low_memory=False)
    d["source_file"] = p.name
    dfs.append(d)
df = pd.concat(dfs, ignore_index=True)

# 2) Basic cleanup
# drop unnamed columns
drop_cols = [c for c in df.columns if c.lower().startswith("unnamed")]
df.drop(columns=drop_cols, inplace=True, errors="ignore")

# parse date if present
for cand in ["date","Date","timestamp","time"]:
    if cand in df.columns:
        df["date"] = pd.to_datetime(df[cand], errors="coerce")
        break

# standardize numeric columns
num_like = ["price","1h","24h","7d","24h_volume","mkt_cap"]
for c in num_like:
    if c in df.columns:
        df[c] = pd.to_numeric(df[c], errors="coerce")

# 3) Target: liquidity_ratio = 24h_volume / mkt_cap
assert "24h_volume" in df.columns and "mkt_cap" in df.columns, "Required columns missing"
df["liquidity_ratio"] = df["24h_volume"] / df["mkt_cap"]
df["liquidity_ratio"].replace([np.inf, -np.inf], np.nan, inplace=True)

print("Shape:", df.shape)
print("Columns:", list(df.columns))
df.head()


Shape: (1000, 11)
Columns: ['coin', 'symbol', 'price', '1h', '24h', '7d', '24h_volume', 'mkt_cap', 'date', 'source_file', 'liquidity_ratio']


C:\Users\krpra\AppData\Local\Temp\ipykernel_18644\165431313.py:37: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df["liquidity_ratio"].replace([np.inf, -np.inf], np.nan, inplace=True)


,coin,symbol,price,1h,24h,7d,24h_volume,mkt_cap,date,source_file,liquidity_ratio
0,Bitcoin,BTC,40859.460000,0.022,0.030,0.055,3.539076e+10,7.709915e+11,2022-03-16,coin_gecko_2022-03-16.csv,0.045903
1,Ethereum,ETH,2744.410000,0.024,0.034,0.065,1.974870e+10,3.271044e+11,2022-03-16,coin_gecko_2022-03-16.csv,0.060374
2,Tether,USDT,1.000000,-0.001,-0.001,0.000,5.793497e+10,7.996516e+10,2022-03-16,coin_gecko_2022-03-16.csv,0.724503
3,BNB,BNB,383.430000,0.018,0.028,0.004,1.395854e+09,6.404382e+10,2022-03-16,coin_gecko_2022-03-16.csv,0.021795
4,USD Coin,USDC,0.999874,-0.001,0.000,-0.000,3.872274e+09,5.222214e+10,2022-03-16,coin_gecko_2022-03-16.csv,0.074150


### Handle Missing Values
We fill missing percent-change fields with 0, drop rows with missing critical values (`price`, `24h_volume`, `mkt_cap`), and clean infinite values in the target.


In [ ]:
import numpy as np

# Fill % change columns with 0 if missing
for pc in ["1h","24h","7d"]:
    if pc in df.columns:
        df[pc] = df[pc].fillna(0)

# Replace inf/-inf in target without inplace (avoids warning)
if "liquidity_ratio" in df.columns:
    lr = df["liquidity_ratio"]
    df["liquidity_ratio"] = lr.where(np.isfinite(lr), np.nan)

# Drop rows missing critical numeric fields
critical = [c for c in ["price","24h_volume","mkt_cap"] if c in df.columns]
before = len(df)
df = df.dropna(subset=critical)
after = len(df)

# Fill any remaining target NaNs with 0 (tiny fraction expected)
if "liquidity_ratio" in df.columns:
    df["liquidity_ratio"] = df["liquidity_ratio"].fillna(0)

print(f"Dropped rows due to critical NA: {before - after}")
df.isna().sum().sort_values(ascending=False).head(10)


### Save Processed Data
We save the cleaned dataset for later use and show a summary.


In [3]:
from pathlib import Path

PROCESSED = BASE/"data"/"processed"
PROCESSED.mkdir(parents=True, exist_ok=True)
out_path = PROCESSED/"merged_coin_gecko.csv"
df.to_csv(out_path, index=False)

summary = {
    "rows": len(df),
    "cols": len(df.columns),
    "date_min": str(df["date"].min()) if "date" in df.columns else None,
    "date_max": str(df["date"].max()) if "date" in df.columns else None,
    "null_counts_top": df.isna().sum().sort_values(ascending=False).head(10).to_dict(),
}
print("Saved:", out_path)
summary

Saved: C:\Users\krpra\Desktop\Project\crypto_liquidity_project\data\processed\merged_coin_gecko.csv


{'rows': 993,
 'cols': 11,
 'date_min': '2022-03-16 00:00:00',
 'date_max': '2022-03-17 00:00:00',
 'null_counts_top': {'coin': 0,
  'symbol': 0,
  'price': 0,
  '1h': 0,
  '24h': 0,
  '7d': 0,
  '24h_volume': 0,
  'mkt_cap': 0,
  'date': 0,
  'source_file': 0}}